In [ ]:
import pandas as pd
import numpy as np
import sqlalchemy as sal
import urllib

In [ ]:
df=pd.read_csv("customer_shopping_behavior.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df['Review Rating']=df.groupby('Category')['Review Rating'].transform(lambda x: x.fillna(x.median()))

In [ ]:
df.isnull().sum()

In [ ]:
df.columns=df.columns.str.lower()
df.columns=df.columns.str.replace(' ','_')
df=df.rename(columns={'purchase_amount_(usd)':'purchase_amount'})

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
def age_label(age):
    if age<=25:
        return 'Young-Adult'
    elif age>25 and age<=40:
        return 'Adult'
    elif age>40 and age<=50:
        return 'Mid-Senior'
    else:
        return 'Senior'
df['age_group1']=df['age'].apply(age_label)

In [ ]:
df['age_group']=df['age_group1']
df.head(10)
df.drop(['age_group1'],axis=1,inplace=True)

In [ ]:
frequency_mapping={
    'Fortnightly':14,
    'Weekly':7,
    'Annually':365,
    'Quarterly':90, 
    'Bi-Weekly':14,
    'Monthly':30,
    'Every 3 Months':90
}
df['purchase_frequency_days']=df['frequency_of_purchases'].map(frequency_mapping)
df.head(10)

In [ ]:


if (df['discount_applied'] == df['promo_code_used']).all():
    df.drop(['promo_code_used'], axis=1, inplace=True)

In [ ]:
df.columns

In [ ]:
# Database connection parameters


server = "tcp:your_server.database.windows.net,1433"
database = "your_database"
username = "your_username"
password = "your_password"
driver = "ODBC Driver 17 for SQL Server"

# Build the ODBC connection string
connection_string = (
    f"DRIVER={{{driver}}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password};"
)


params = urllib.parse.quote_plus(connection_string)

engine = sal.create_engine(f"mssql+pyodbc:///?odbc_connect={params}")


In [ ]:
# Write the DataFrame to SQL Server

df.to_sql(
    name="customer_shopping_behavior",
    con=engine,
    if_exists="replace",
    index=False
)
